# Query from NYC OpenData

In [3]:
import pandas as pd
import requests
import urllib.parse

1. Go to https://data.cityofnewyork.us/login, create an account, and sign in
2. Browse to 'Your Profile' --> Edit (pencil icon) --> Developer Settings
3. Click 'Create New App Token'
4. Copy the App Token (not Secret Token) and paste it below:

In [ ]:
OPEN_DATA_APP_TOKEN = 

In [5]:
BASE_URL = 'https://data.cityofnewyork.us/'

The table id is the 4-4 digit code in the table url after the category and name

e.g.:

[https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/**erm2-nwe9**/about_data](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/about_data)

In [6]:
table_id = 'erm2-nwe9'

The query can be any be any Socrata-flavor SQL, including any of [these functions](https://dev.socrata.com/docs/functions/#,).

If you do not include a `LIMIT`, by default the query will be truncated at 1000 rows

In [7]:
query = """
    SELECT
        DATE_TRUNC_YMD(`created_date`) as date,
        COUNT(`unique_key`) as count
    WHERE
        (`complaint_type` LIKE "Homeless Person Assistance")
        AND (`created_date` >= "2019-01-01")
    GROUP BY 
        DATE_TRUNC_YMD(`created_date`)
    LIMIT 100000
    """

In [8]:
data_request_url = (
    BASE_URL
    + 'resource/'
    + table_id
    +'.json?'
    + '$query='
    + urllib.parse.quote(query)
    + '&$$app_token='
    + OPEN_DATA_APP_TOKEN
)

In [9]:
data_request_url

'https://data.cityofnewyork.us/resource/erm2-nwe9.json?$query=%0A%20%20%20%20SELECT%0A%20%20%20%20%20%20%20%20DATE_TRUNC_YMD%28%60created_date%60%29%20as%20date%2C%0A%20%20%20%20%20%20%20%20COUNT%28%60unique_key%60%29%20as%20count%0A%20%20%20%20WHERE%0A%20%20%20%20%20%20%20%20%28%60complaint_type%60%20LIKE%20%22Homeless%20Person%20Assistance%22%29%0A%20%20%20%20%20%20%20%20AND%20%28%60created_date%60%20%3E%3D%20%222019-01-01%22%29%0A%20%20%20%20GROUP%20BY%20%0A%20%20%20%20%20%20%20%20DATE_TRUNC_YMD%28%60created_date%60%29%0A%20%20%20%20LIMIT%20100000%0A%20%20%20%20&$$app_token=xxxx'

In [ ]:
r = requests.get(data_request_url)

data = pd.DataFrame(r.json())